In [3]:
import ctypes
from ctypes import *
from ctypes.util import find_library


# Setting language of OCR
lang = b"eng"

# File path of image from which text needs to be extracted
filename = "//home//kgplatformuser//python//ocr//samples//Results//PDFImages//PO10_Cropped_CroppedExPgsRmvd_1.png"

# file path of tessdata
TESSDATA_PREFIX = b"//usr//share//tesseract-ocr//4.00//"

# file path of libtesseract with version 4
path = "//usr//lib//libtesseract.so.4"


In [4]:
# makes a call to libtesseract
tesseract = CDLL(path)

class TessBaseAPI(Structure):
    pass
class TessResultRenderer(Structure):
    pass


# verifying tesseract version
tesseract.TessVersion.restype = ctypes.c_char_p
tesseract_version = tesseract.TessVersion()
print(tesseract_version)
print(tesseract)

#C++ Config Parameters
# I am not sure what this code does. It seems like it is defining the pointers that are required for C++. Need to look into this
tesseract.TessBaseAPICreate.restype = POINTER(TessBaseAPI)
tesseract.TessBaseAPIInit3.argtypes = [POINTER(TessBaseAPI), c_char_p, c_char_p]
tesseract.TessBaseAPIInit3.restype = c_bool
tesseract.TessBaseAPIProcessPages.argtypes = [POINTER(TessBaseAPI), c_char_p, c_char_p, c_int, POINTER(TessResultRenderer)]
tesseract.TessBaseAPIProcessPages.restype = c_bool
tesseract.TessBaseAPIGetUTF8Text.argtypes = [POINTER(TessBaseAPI)]
tesseract.TessBaseAPIGetUTF8Text.restype = c_char_p


4.00.00alpha
<CDLL '//usr//lib//libtesseract.so.4', handle 28da590 at 7f18943db610>


In [5]:
# creates a new instance of TessBaseAPI
api = tesseract.TessBaseAPICreate()

# intitializes the api instance with the trained data and language
rc = tesseract.TessBaseAPIInit3(api, TESSDATA_PREFIX, lang);
if (rc):
    tesseract.TessBaseAPIDelete(api)
    print("Could not initialize tesseract.\n",path)
    exit(3)


In [6]:
#Load Tesseract Page Segmentation Mode 
    
# the following line sets the page segmentation mode:
# PSM_AUTO	3 Fully automatic page segmentation, but no OSD.
# PSM_AUTO_ONLY	2 Automatic page segmentation, but no OSD, or OCR.
# PSM_AUTO_OSD	1 Automatic page segmentation with orientation and script detection.
# PSM_CIRCLE_WORD	9 Treat the image as a single word in a circle.
# PSM_COUNT	13 Number of enum entries.
# PSM_OSD_ONLY	0 Orientation and script detection only.
# PSM_SINGLE_BLOCK	6 Assume a single uniform block of tex
# PSM_SINGLE_BLOCK_VERT_TEXT	5 Assume a single uniform block of vertically aligned text.
# PSM_SINGLE_CHAR	10 Treat the image as a single character.
# PSM_SINGLE_COLUMN	4 Assume a single column of text of variable sizes.
# PSM_SINGLE_LINE	7 Treat the image as a single text line.
# PSM_SINGLE_WORD	8 Treat the image as a single word.
# PSM_SPARSE_TEXT	11 Find as much text as possible in no particular order.
# PSM_SPARSE_TEXT_OSD	12 Sparse text with orientation and script detection.

#PSM 1 Is great at handling text with both lower / upper and special characters 
tesseract.TessBaseAPISetPageSegMode(api,1)

#PSM 3 is Default 
#tesseract.TessBaseAPISetPageSegMode(api,3)

#PSM 6 is great for tables 
#tesseract.TessBaseAPISetPageSegMode(api,6)

#print current page segmentation mode 
print(tesseract.TessBaseAPIGetPageSegMode(api))


1


In [7]:
# the following line starts processing the image
success = tesseract.TessBaseAPIProcessPages(api, filename, None , 0, None)
success

True

In [ ]:
# the following line runs page analysis in the pagesegmode set above and returns a TessPageIterator
iterator = tesseract.TessBaseAPIAnalyseLayout(api)
if(iterator is not None):
    print ("1")
    print(iterator)
    
# Resets the iterator to point to the start of the page.       
tesseract.TessPageIteratorBegin(iterator)  

# checks if iterator is at the start of the given block level
if(tesseract.TessPageIteratorIsAtBeginningOf(iterator,0)):
    print("yes")


1
137800960
yes


In [ ]:
# gets the block type of current block of iterator
# PT_CAPTION_TEXT	8 Text that belongs to an image.
# PT_COUNT	15 Number of enum entries.
# PT_EQUATION	4 Partition belonging to an equation region.
# PT_FLOWING_IMAGE	9 Image that lives inside a column.
# PT_FLOWING_TEXT	1 Text that lives inside a column.
# PT_HEADING_IMAGE	10 Image that spans more than one column.
# PT_HEADING_TEXT	2 Text that spans more than one column.
# PT_HORZ_LINE	12 Horizontal Line.
# PT_INLINE_EQUATION	5 Partition has inline equation.
# PT_NOISE	14 Lies outside of any column.
# PT_PULLOUT_IMAGE	11 Image that is in a cross-column pull-out region.
# PT_PULLOUT_TEXT	3 Text that is in a cross-column pull-out region.
# PT_TABLE	6 Partition belonging to a table region.
# PT_UNKNOWN	0 Type is not yet known.
# PT_VERT_LINE	13 Vertical Line.
# PT_VERTICAL_TEXT	7 Text-line runs vertically.
print("blocktype",tesseract.TessPageIteratorBlockType(iterator))


# looping through iterator at block level specified
# RIL_BLOCK	0 Block of text/image/separator line
# RIL_PARA	1 Paragraph within a block
# RIL_SYMBOL	4 Symbol/character within a word.
# RIL_TEXTLINE	2 Line within a paragraph.
# RIL_WORD	3 Word within a textline.
while(tesseract.TessPageIteratorNext(iterator,0)):
    print("2")

# need to see if this gives list of identified blocks. need to explore further.
blocklist = tesseract.TessBaseAPIFindLinesCreateBlockList(api)
print(blocklist)
 
    

if success:
     print("success")
     text = tesseract.TessBaseAPIGetUTF8Text(api)
     print("="*78)
     print(text.decode("utf-8").strip())
     print("="*78)
else:
     print("fail")